# Basic

In [19]:
# Basic HW ES
# Working out the formula into code

import numpy as np
import time

# Read data 
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)

# List to store execution times
number_of_executions = 1
execution_times = []

# Smoothing factors
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24  # Season = 1 Day for 1 value per hour

# Holt-Winters
for i in range(number_of_executions):

    # Initialization or reset of parameter arrays
    n = len(data)
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(n)
    smoothed = np.zeros(n)

    # Assign initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    # Start teh timer
    start_time = time.time()

    # Holt-Winters
    for t in range(1, n):

        # Level: a[t] = α * (y[t] − s[t−j]) + (1 − α) * (a[t−1] + b[t−1])   
        level[t] = alpha * (data[t] - season[t-season_length]) + (1 - alpha) * (level[t-1] + trend[t-1])

        # Trend: b[t] = β * (a[t] − a[t−1]) + (1 − β) * b[t−1]
        trend[t] = beta * (level[t] - level[t-1]) + (1 - beta) * trend[t-1]

        # Season: s[t] = γ * (y[t] − a[t]) + (1 − γ) * s[t−j]
        if t >= season_length:
            season[t] = gamma * (data[t] - level[t]) + (1 - gamma) * season[t-season_length]

        # Forecast: Y[t,h] = a[t] + b[t] + s[t−j+m]
        smoothed[t] = level[t] + trend[t] + season[t-t%season_length]

        # Print intermediate values for debugging
        #print(f"t={t}, level={level[t]}, trend={trend[t]}, season={season[t]}, smoothed={smoothed[t]}")
        #print(f"t={t}, level={level[t]}")

      
    # Stop the timer
    end_time = time.time()
    function_time = end_time - start_time
    execution_times.append(function_time)
    #print(function_time)

# Calculate the function time
function_time = np.mean(execution_times)

# Print results
print("\n### Dataset energy_generation_solar")
print("### Holt-Winters\n")
#print("The smoothed values for energy_generation_solar are:", smoothed)

# Print last smoothed value
smoothed_value = smoothed[-1]
print("The last smoothed value for energy_generation_solar is:", smoothed_value)
print(f"The function was executed in {function_time} seconds")


### Dataset energy_generation_solar
### Holt-Winters

The last smoothed value for energy_generation_solar is: 90.60914407238147
The function was executed in 0.13061308860778809 seconds


# Vectorized

In [7]:
import numpy as np
import time

def holt_winters_smoothing_v8(data, alpha, beta, gamma, season_length):
    n = data.shape[0]

    # Initialize arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(season_length)
    smoothed = np.zeros(n)

    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    # Vectorized approach for each season
    for season_start in range(0, n, season_length):
        season_end = min(season_start + season_length, n)
        season_indices = np.arange(season_start, season_end)
        
        # Calculate level
        level_weights = alpha * (1 - alpha) ** np.arange(season_end - season_start)
        level_smoothed = np.cumsum(level_weights * data[season_indices][::-1])[::-1]
        level[season_indices] = level_smoothed / level_weights.sum()

        # Calculate trend
        trend_weights = beta * (1 - beta) ** np.arange(season_end - season_start)
        trend_smoothed = np.cumsum(trend_weights * (level_smoothed - np.roll(level_smoothed, 1))[::-1])[::-1]
        trend[season_indices] = trend_smoothed / trend_weights.sum()

        # Calculate season
        if season_start == 0:
            season_weights = np.zeros(season_end - season_start)
            season_smoothed = np.zeros(season_end - season_start)
        else:
            season_weights = gamma * (1 - gamma) ** np.arange(season_end - season_start)
            season_smoothed = np.cumsum(season_weights * (data[season_indices] - level_smoothed)[::-1])[::-1]
            season[season_indices % season_length] = season_smoothed / season_weights.sum()

        smoothed[season_indices] = level[season_indices] + trend[season_indices] + season[season_indices % season_length]

        # Print intermediate values for debugging
        #print(f"Season start: {season_start}, Season end: {season_end}")
        print(f"Level: {level[season_indices]}")
        #print(f"Trend: {trend[season_indices]}")
        #print(f"Season: {season[season_indices % season_length]}")
        #print(f"Smoothed: {smoothed[season_indices]}")

    return smoothed

# Example usage
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24

start_time = time.time()
smoothed = holt_winters_smoothing_v8(data, alpha, beta, gamma, season_length)
end_time = time.time()

# Print the results
print("### Dataset wind_speed")
print("### #2 Vectorized \n")
print("The last smoothed value for wind_speed is:", smoothed[-1])
print(f"The function was executed in {end_time - start_time} seconds")

Level: [612.62034147 612.56103181 612.48689473 612.39422338 612.29691847
 612.19845516 612.07537602 611.91700214 611.61156678 606.35836153
 588.51478582 553.19661372 499.54823298 430.38186824 344.65752112
 241.6825786  137.57485778  75.75247013  54.47073477  45.31800824
  37.08713906  29.37069921  19.85375672  10.85124355]
Level: [705.48521709 705.42116266 705.34109461 705.24286298 705.12007345
 704.93762673 704.73852813 704.49417985 703.83805946 696.49205642
 675.51988942 634.65597995 577.29294943 506.97007197 423.87808588
 326.27035488 224.32030702 137.80267795  97.29363219  79.77834257
  66.19740842  52.10204495  34.48284061  12.45883519]
Level: [833.39049889 833.31695491 833.22650767 833.11344863 832.97212482
 832.79547006 832.57465161 832.28957862 830.97168163 825.17406615
 810.45908171 782.97343512 738.53572876 674.75491575 593.19487746
 502.30290358 411.54578426 330.16948531 276.70175911 243.64658848
 214.42700289 184.48721625 157.86549874 139.05667659]
Level: [1878.25327107 187

# Library

In [16]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Example dataset
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/replicated_preprocessed_data_files/energy_generation_solar.csv', delimiter=',', skip_header=1)


# Fit the model
model = ExponentialSmoothing(data, seasonal_periods=24, trend='add', seasonal='add', initial_level=(data[0]), initial_trend=(data[1] - data[0]), initial_seasonal=(data[:season_length] - level[0]), initialization_method='known')
fit_model = model.fit(smoothing_level=0.2, smoothing_trend=0.1, smoothing_seasonal=0.3, optimized=False)

# Get the fitted values (smoothed values)
smoothed_values = fit_model.fittedvalues
print(smoothed_values)

# Get the last smoothed value
last_smoothed_value = smoothed_values[-1]
print("The last smoothed value is:", last_smoothed_value)

[ 50.          50.          50.         ... 581.76011552 510.74993221
 442.92321108]
The last smoothed value is: 442.9232110827344


In [18]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Example dataset
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/replicated_preprocessed_data_files/energy_generation_solar.csv', delimiter=',', skip_header=1)


# Fit the model
model = ExponentialSmoothing(data, seasonal_periods=24, trend='add', seasonal='add', initialization_method='heuristic')
fit_model = model.fit(smoothing_level=0.2, smoothing_trend=0.1, smoothing_seasonal=0.3, optimized=False)

# Get the fitted values (smoothed values)
smoothed_values = fit_model.fittedvalues
print(smoothed_values)

# Get the last smoothed value
last_smoothed_value = smoothed_values[-1]
print("The last smoothed value is:", last_smoothed_value)

[-78.71526989 -25.58878804   5.23146487 ... 581.76011552 510.74993221
 442.92321108]
The last smoothed value is: 442.92321108273484


___
## Double Exponential Smoothing Test

In [97]:
# LIBRARY

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Example dataset
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/replicated_preprocessed_data_files/energy_generation_solar.csv', delimiter=',', skip_header=1)
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/wind_speed_small.csv', delimiter=',', skip_header=1)
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/replicated_preprocessed_data_files/wind_speed.csv', delimiter=',', skip_header=1)
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/heart_rate_small.csv', delimiter=',', skip_header=1)




# Fit the model
model = ExponentialSmoothing(data, trend='add')
fit_model = model.fit(smoothing_level=0.2, smoothing_trend=0.1, optimized=False)

# Get the fitted values (smoothed values)
smoothed_values = fit_model.fittedvalues
print(smoothed_values)

# Get the last smoothed value
last_smoothed_value = smoothed_values[-1]
print("The last smoothed value is:", last_smoothed_value)

[ 93.83636364  94.84872727  96.68164364 ... 116.53664821 113.56989265
 110.30509035]
The last smoothed value is: 110.30509034850527


In [98]:
# Basic

import numpy as np
import time

def double_exponential_smoothing(data, alpha, beta):
    n = len(data)

    # Initialize arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    smoothed = np.zeros(n)
    
    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]

    # Holt-Winters without seasonal component
    for t in range(1, n):
        # Level: a[t] = α * y[t] + (1 - α) * (a[t−1] + b[t−1])
        level[t] = alpha * data[t] + (1 - alpha) * (level[t-1] + trend[t-1])
        # Trend: b[t] = β * (a[t] − a[t−1]) + (1 - β) * b[t−1]
        trend[t] = beta * (level[t] - level[t-1]) + (1 - beta) * trend[t-1]
        # Final smoothed values
        smoothed[t] = level[t] + trend[t]

    return smoothed

#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/replicated_preprocessed_data_files/wind_speed.csv', delimiter=',', skip_header=1)
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/wind_speed_small.csv', delimiter=',', skip_header=1)
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/heart_rate_small.csv', delimiter=',', skip_header=1)

alpha = 0.2
beta = 0.1

start_time = time.time()
smoothed = double_exponential_smoothing(data, alpha, beta)
end_time = time.time()
print(smoothed)

# Print the results
print("### Dataset wind_speed")
print("### Double Exponential Smoothing \n")
print("The last smoothed value for wind_speed is:", smoothed[-2])
print(f"The function was executed in {end_time - start_time} seconds")


[  0.         101.         105.34       ... 113.56989265 110.30509035
 107.2471467 ]
### Dataset wind_speed
### Double Exponential Smoothing 

The last smoothed value for wind_speed is: 110.30509034850527
The function was executed in 1.6628139019012451 seconds


In [96]:
# Vectorized 

import numpy as np
import time

def double_exponential_smoothing(data, alpha, beta):
    n = data.shape[0]

    # Initialize level and trend arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    
    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    
    # Calculate level component 
    weights_level = np.array([alpha * (1 - alpha) ** np.arange(n)])
    cum_weights_level = np.cumsum(weights_level)
    level = np.cumsum(weights_level * data[::-1])[::-1] / cum_weights_level[::-1]

    # Calculate trend component 
    weights_trend = np.array([beta * (1 - beta) ** np.arange(n-1)])
    cum_weights_trend = np.cumsum(weights_trend)
    trend_diff = level[1:] - level[:-1]
    trend = np.cumsum(weights_trend * trend_diff[::-1])[::-1] / cum_weights_trend[::-1]
    #trend = np.concatenate(([0], trend))  # Align trend with the original data
    level = level[1:]

    # Final smoothed values
    smoothed = level + trend
    
    return smoothed

#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/wind_speed_small.csv', delimiter=',', skip_header=1)
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/replicated_preprocessed_data_files/wind_speed.csv', delimiter=',', skip_header=1)
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/heart_rate_small.csv', delimiter=',', skip_header=1)


alpha = 0.2
beta = 0.1
start_time = time.time()
smoothed = double_exponential_smoothing(data, alpha, beta)
end_time = time.time()
print(smoothed)

# Print the results
print("### Dataset energy_generation_solar")
print("### Double Exponential Smoothing \n")
print("The last smoothed value for energy_generation_solar is:", smoothed[-1])
print(f"The function was executed in {end_time - start_time} seconds")


[105.56619194 105.56619194 105.56619194 ...  98.45630817  97.76876618
  97.55555556]
### Dataset energy_generation_solar
### Double Exponential Smoothing 

The last smoothed value for energy_generation_solar is: 97.55555555555556
The function was executed in 0.07874107360839844 seconds


In [99]:
# Vectorized 

import numpy as np
import time

def double_exponential_smoothing(data, alpha, beta):
    n = data.shape[0]

    # Initialize level and trend arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    
    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    
    # Calculate level component 
    weights_level = np.array(alpha * (1 - alpha) ** np.arange(n))
    cum_weights_level = np.cumsum(weights_level)
    level = np.cumsum(weights_level * data[::-1])[::-1] / cum_weights_level[::-1]


    # Calculate trend component 
    weights_trend = np.array(beta * (1 - beta) ** np.arange(n))
    cum_weights_trend = np.cumsum(weights_trend)
    trend_diff = level[1:] - level[:-1]

    trend = np.cumsum(weights_trend[:trend_diff.shape[0]] * trend_diff[::-1])[::-1]
    trend = trend / cum_weights_trend[:trend_diff.shape[0]][::-1]

    # Final smoothed values
    smoothed = level 
    smoothed[1:] += trend
    
    return smoothed

# Example usage
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/wind_speed_small.csv', delimiter=',', skip_header=1)
#data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/replicated_preprocessed_data_files/wind_speed.csv', delimiter=',', skip_header=1)
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/heart_rate_small.csv', delimiter=',', skip_header=1)


alpha = 0.2
beta = 0.1
start_time = time.time()
smoothed = double_exponential_smoothing(data, alpha, beta)
end_time = time.time()
print(smoothed)

# Print the results
print("### Dataset energy_generation_solar")
print("### Double Exponential Smoothing \n")
print("The last smoothed value for energy_generation_solar is:", smoothed[-1])
print(f"The function was executed in {end_time - start_time} seconds")


[106.11992735 105.56619194 105.56619194 ...  98.45630817  97.76876618
  97.55555556]
### Dataset energy_generation_solar
### Double Exponential Smoothing 

The last smoothed value for energy_generation_solar is: 97.55555555555556
The function was executed in 0.0815889835357666 seconds


___
___
___

In [ ]:
import numpy as np

def cumsumprod(X, C, start):
    """
    Computes the cumulative sum product in a vectorized manner.
    
    Parameters:
    X (numpy array): Input array X.
    C (numpy array): Input array C.
    start (float): Initial start value.
    
    Returns:
    numpy array: Resulting array Y.
    """
    Y = np.copy(X)
    P = np.copy(C)
    m = X.shape[0]
    Y[0] = X[0] + C[0] * start

    k = 1
    while k < m:
        Y[k:m] = Y[k:m] + Y[0:m-k] * P[k:m]
        P[k:m] = P[0:m-k] * P[k:m]
        k *= 2
        
    return Y

def double_exponential_smoothing(data, alpha, beta):
    """
    Perform double exponential smoothing (Holt's Linear Trend Model) on the given time series data.
    
    Parameters:
    data (numpy array): The time series data to be smoothed.
    alpha (float): The smoothing factor for the level component (0 < alpha < 1).
    beta (float): The smoothing factor for the trend component (0 < beta < 1).
    
    Returns:
    numpy array: The smoothed time series data.
    """
    n = len(data)
    level = np.zeros(n)
    trend = np.zeros(n)
    
    level[0] = data[0]
    trend[0] = data[1] - data[0]

    level_X = alpha * data
    level_C = np.ones(n) * (1 - alpha)
    trend_X = beta * (level - np.roll(level, 1))
    trend_C = np.ones(n) * (1 - beta)
    
    level = cumsumprod(level_X, level_C, level[0])
    trend = cumsumprod(trend_X, trend_C, trend[0])
    
    smoothed = level + trend
    
    return smoothed

# Example usage
data = np.array([3, 10, 12, 13, 12, 10, 12, 16, 19, 20])
alpha = 0.8
beta = 0.2
smoothed_data = double_exponential_smoothing(data, alpha, beta)
print(smoothed_data)


In [ ]:
import numpy as np

def cumsumprod(X, C, start):
    """
    Computes the cumulative sum product in a vectorized manner.
    
    Parameters:
    X (numpy array): Input array X.
    C (numpy array): Input array C.
    start (float): Initial start value.
    
    Returns:
    numpy array: Resulting array Y.
    """
    Y = np.copy(X)
    P = np.copy(C)
    m = X.shape[0]
    Y[0] = X[0] + C[0] * start

    k = 1
    while k < m:
        Y[k:m] = Y[k:m] + Y[0:m-k] * P[k:m]
        P[k:m] = P[0:m-k] * P[k:m]
        k *= 2
        
    return Y

def triple_exponential_smoothing(data, alpha, beta, gamma, season_length):
    """
    Perform triple exponential smoothing (Holt-Winters Method) on the given time series data.
    
    Parameters:
    data (numpy array): The time series data to be smoothed.
    alpha (float): The smoothing factor for the level component (0 < alpha < 1).
    beta (float): The smoothing factor for the trend component (0 < beta < 1).
    gamma (float): The smoothing factor for the seasonal component (0 < gamma < 1).
    season_length (int): The length of the seasonality cycle.
    
    Returns:
    numpy array: The smoothed time series data.
    """
    n = len(data)
    level = np.zeros(n)
    trend = np.zeros(n)
    seasonality = np.zeros(n)
    
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    seasonality[:season_length] = data[:season_length] - level[0]
    
    level_X = alpha * (data - seasonality)
    level_C = np.ones(n) * (1 - alpha)
    trend_X = beta * (level - np.roll(level, 1))
    trend_C = np.ones(n) * (1 - beta)
    season_X = gamma * (data - level)
    season_C = np.ones(n) * (1 - gamma)
    
    level = cumsumprod(level_X, level_C, level[0])
    trend = cumsumprod(trend_X, trend_C, trend[0])
    seasonality = cumsumprod(season_X, season_C, seasonality[0])
    
    smoothed = level + trend + seasonality
    
    return smoothed

# Example usage
data = np.array([3, 10, 12, 13, 12, 10, 12, 16, 19, 20])
alpha = 0.8
beta = 0.2
gamma = 0.1
season_length = 4
smoothed_data = triple_exponential_smoothing(data, alpha, beta, gamma, season_length)
print(smoothed_data)
